# turns all the relevant markdown files in chris albon into bricks. No template needed for this

In [12]:
import subprocess
import re
import json
from os import walk

In [13]:
markdown = ['linux/basics', 'linux/environment', 'linux/inputs_and_outputs', 'linux/search', 'linux/text', 
            'postgresql/add_delete_change', 'postgresql/basics', 'postgresql/dates', 'postgresql/merging_and_joining',
            'postgresql/numeric', 'postgresql/tables', 'postgresql/text']


In [14]:
class Snippet():
    def __init__(self, full_path, tag_path, file):
        self.full_path = full_path
        self.tag_path = tag_path
        self.file = file
        self.setTags()
        self.getMarkdown()
        self._setProperties()
        self.name = self.title.split('\"')[1]

        
    def setTags(self):
        tags = self.tag_path.split('/')
        if tags[-1] in ['basics','other']:
            tags = [': '.join(tags)]
        self.tags = tags
        
    def getMarkdown(self):
        with open(self.full_path +'/'+ self.file, 'r') as myfile:
            self.lines = myfile.readlines()
            self.markdown = ''.join(self.lines[9:]).replace(
                '{{< highlight sql >}}', '```sql').replace('{{< /highlight >}}', '```').replace(
                '{{< highlight markdown >}}', '```')
    
    def _setProperties(self):
        self.title = self.lines[1]
        self.desc = self.lines[4]
            
    def _firstcell(self):
        markdown_cell = {'cell_type': 'markdown',
            'metadata': {},
            'source': ['---\n',
            self.title,
            self.desc,
            'tags: '+ ', '.join(self.tags)+'\n',
            'URL: https://github.com/ActiveState/code\n',
            #'Licence: '+self.copyright+'\n',
            'Creator: \n',
            'Meta: \n',
            '\n',
            '---']}

        return markdown_cell
    
    def _nextCell(self):
        markdown_cell = {'cell_type': 'markdown',
            'metadata': {},
            'source': [self.markdown]
            }

        return markdown_cell
    

    def write_to_file(self):
        s = '''{
                 "cells": [{
                   "cell_type": "markdown",
                   "metadata": {},
                   "source": ["# Example&"]
                  }],
                 "metadata": {
                  "kernelspec": {
                   "display_name": "Python 3",
                   "language": "python",
                   "name": "python3"
                  },
                  "language_info": {
                   "codemirror_mode": {
                    "name": "ipython",
                    "version": 3
                   },
                   "file_extension": ".py",
                   "mimetype": "text/x-python",
                   "name": "python",
                   "nbconvert_exporter": "python",
                   "pygments_lexer": "ipython3",
                   "version": "3.7.0"
                  }
                 },
                 "nbformat": 4,
                 "nbformat_minor": 2
                }
                '''
        data = json.loads(s)
        data['cells'].insert(0, self._firstcell())
        data['cells'].insert(2, self._nextCell())

        with open ('./inprogress/'+str(self.name)+'.ipynb', 'w') as outfile:
            json.dump(data, outfile)

In [15]:
mypath = '/Users/jimmydelano/notes/content/'

snippet_list = []
for (dirpath, dirnames, filenames) in walk(mypath):
    if dirpath[len(mypath):] in markdown:
        for file in filenames:
            if file.endswith('.md'):
                clean_dir = dirpath.replace(mypath,'')
                s = Snippet(dirpath, clean_dir, file)
                snippet_list.append(s)
          

In [16]:
for s in snippet_list:
    s.write_to_file()